In [ ]:
# Times Square parameters
day = "2024-11-17"
start_hour = "06:00"

# Weather Forecast: Temperature Outside the Dome

Author: Johnny Esteves

This notebook contains forecast information for the temperature outside the dome. You will find a forecast for a given date and hour of what should be the temperature at the nautical twilight.


The details of each section are explained below. 

In [ ]:
# Standard Library Imports
import warnings
from datetime import UTC, datetime, timedelta

# Third-Party Imports
import numpy as np
import pandas as pd
from astropy.time import Time
from pytz import timezone
from astroplan import Observer
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# Local Imports
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

In [ ]:
# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

### Nautical Twilight

Today's information about the nautical twilight.

In [ ]:
# Establish the timeframe for sun set/rise for the night

local_timezone = timezone("America/Santiago")
timestamp_format = "%Y-%m-%d %H:%M:%S %Z"
auxtel_site = Observer.at_site("Rubin AuxTel")

# Datetimes to establish reference points for finding sunset times
# and measurement times for moon illumination
threeAM_local_datetime = datetime.fromisoformat(day).replace(
    hour=3, minute=0, second=0, tzinfo=local_timezone
)

# Astropy Time for astroplan calculations
threeAM_time = Time(threeAM_local_datetime, scale="utc")

# Compute sunrise/sunset
sunset_time = auxtel_site.sun_set_time(threeAM_time, which="next")
sunset_datetime = sunset_time.to_datetime(timezone=UTC)
sunset_local_datetime = sunset_time.to_datetime(timezone=local_timezone)

sunrise_time = auxtel_site.sun_rise_time(threeAM_time, which="next")
sunrise_datetime = sunrise_time.to_datetime(timezone=UTC)

# Nautical sunset and sunrise (preferred by observers over astronomical twilight)
evening_twilight_datetime = auxtel_site.twilight_evening_nautical(
    threeAM_time, which="next"
).to_datetime(timezone=UTC)
evening_twilight_local_datetime = auxtel_site.twilight_evening_nautical(
    threeAM_time, which="next"
).to_datetime(timezone=local_timezone)

evening_twilight = Time(evening_twilight_datetime)

morning_twilight_datetime = auxtel_site.twilight_morning_nautical(
    evening_twilight, which="next"
).to_datetime(timezone=UTC)

In [ ]:
print(5 * "-------")
print("Forecast Temperature of the Day")
print("Date of Forecast: ", day)
print("Civic Twilight Local Time: ", sunset_local_datetime)
print("Nautical Twilight Local Time: ", evening_twilight_local_datetime)
print(5 * "-------")

In [ ]:
# Create an EFD client
client = makeEfdClient()

end_time = Time(evening_twilight + timedelta(hours=6.0))
start_time = Time(end_time - timedelta(days=3.0) - timedelta(hours=6.0))

print(
    f"\nQuery data for {day}"
    f"\n  starts at {start_time.isot} and"
    f"\n  ends at {end_time.isot}\n"
)

In [ ]:
df_outside = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_outside.salIndex == 301
df_outside = df_outside[mask]

# Print the location of this sensor
print("Sensor location: ", df_outside["location"].unique())

# We do not need the salIndex anymore
df_outside = df_outside.drop(columns=["salIndex"])

In [ ]:
# Get the rolling min/mean/max values for the temperature
df_outside = df_outside.rename(columns={"temperatureItem0": "temperature"})
df_outside = df_outside.rolling("30min").agg({"temperature": ["min", "mean", "max"]})

df_outside.columns = df_outside.columns.droplevel(0)
df_outside = df_outside.resample("30min").mean()
# df_outside.columns = df_outside.columns.droplevel(0)

## Forecast: Fourier Decomposition Method


In [ ]:
## Define Forecast Model
## Season + Trend
## Trend: 2-order polynomial fit
## Season: 2-fourrier component model


class FourierFit:
    def __init__(self, df, start_date, ycol="mean_sub", frequencies=None):
        """
        Initialize the FourierFit model.

        Parameters:
        - df: DataFrame containing the time series data.
        - start_date: Timestamp indicating when to start predictions (splits the data).
        - ycol: Column name of the target variable for fitting and prediction (default 'mean').
        - frequencies: List of frequencies (in hours) for Fourier components. If None, defaults will be used.
        """
        self.start_date = (
            pd.Timestamp(start_date).tz_localize("America/Santiago")
            if pd.Timestamp(start_date).tzinfo is None
            else pd.Timestamp(start_date)
        )
        self.frequencies = frequencies if frequencies is not None else [24, 12, 8]

        # get only the last 3 days
        self.df = df[df.index >= self.start_date - pd.Timedelta(2.7, "d")].copy()
        self.ycol = ycol

        self.params = None

        # Split data based on start_date
        self.df_fit = self.df.copy()  # [self.df.index < self.start_date].copy()
        self.df_pred = self.df[self.df.index >= self.start_date].copy()

    def find_dominant_frequencies(self, num_components=5, max_frequency=32):
        t_days = (self.df_fit.index - self.df_fit.index[0]).total_seconds() / 3600
        y_data = self.df_fit[self.ycol].values

        N = len(t_days)
        yf = fft(y_data - np.mean(y_data))  # Center data
        xf = 1 / fftfreq(N, np.median(np.diff(t_days)))[: N // 2]

        magnitudes = np.abs(yf[: N // 2])
        sorted_indices = np.argsort(magnitudes)[-num_components * 2 :]

        filtered_freqs = np.where(
            xf[sorted_indices] > max_frequency, -xf[sorted_indices], xf[sorted_indices]
        )
        filtered_freqs = np.sort(-1 * filtered_freqs)[:num_components]

        self.frequencies = np.abs(filtered_freqs)
        print("Identified dominant frequencies (periods ≥ 2 days):", self.frequencies)

    def model_func(self, t, *params):
        c0, c1, c2, c3, c4 = params[:5]
        result = c0 + c1 * t + c2 * t**2  # + c3 * t**3 + c4 * t**4
        offset = 3

        for i, freq in enumerate(self.frequencies):
            a = params[offset + 2 * i]
            b = params[offset + 2 * i + 1]
            result += a * np.cos(2 * np.pi * t / freq) + b * np.sin(
                2 * np.pi * t / freq
            )

        return result

    def fit_model(self):
        t_fit_hours = (self.df_fit.index - self.df_fit.index[0]).total_seconds() / 3600
        y_fit_data = self.df_fit[self.ycol].values
        # initial_guess =  [1, 1] * len(self.frequencies)
        initial_guess = [np.mean(y_fit_data), 0, 0, 0, 0] + [1, 1] * len(
            self.frequencies
        )

        self.params, _ = curve_fit(
            self.model_func, t_fit_hours, y_fit_data, p0=initial_guess
        )

    def forecast_end_hour(self, date_to_forecast):
        """
        Perform an hourly forecast from start_date up to hour_end.
        Store only the forecasted value for each end hour.
        """
        if self.params is None:
            raise Exception("The model must be fitted before forecasting.")

        self.nautical = date_to_forecast
        predictions = []
        current_time = self.start_date
        current_time_hm = current_time.hour + current_time.minute / 60.0
        end_time_hm = date_to_forecast.hour + date_to_forecast.minute / 60.0

        self.df[self.ycol].interpolate(method="time")
        try:
            # actual = yInt.asof(date_to_forecast)
            mask = self.df.index > date_to_forecast - timedelta(hours=0.5)
            mask &= self.df.index < date_to_forecast + timedelta(hours=0.5)
            actual = float(self.df[self.ycol].loc[mask].mean())
        except:
            actual = np.nan

        # Forecast at each hour up to hour_end
        while (current_time_hm - 0.5) < end_time_hm:
            # Add the new data point to df_fit for refitting
            self.df_fit = self.df[self.df.index <= current_time].copy()
            self.fit_model()  # Refit the model with the updated df_fit

            # Calculate the forecast time in hours
            current_time_hours = (
                current_time - self.df_fit.index[0]
            ).total_seconds() / 3600
            forecast_hours = int(end_time_hm - current_time_hm)

            # Predict using the Fourier model for end hour
            predicted_value = self.model_func(
                current_time_hours + forecast_hours, *self.params
            )

            # Store prediction
            predictions.append((current_time, predicted_value))

            # Increment by 1 hour
            current_time += timedelta(hours=0.5)
            current_time_hm += 0.5

        # Create DataFrame of predictions with time index and actual value
        forecast_df = pd.DataFrame(
            {
                "prediction": [val for _, val in predictions],
                "actual": [actual for _ in range(len(predictions))],
            },
            index=[time for time, _ in predictions],
        )

        forecast_df["residual"] = forecast_df["prediction"] - forecast_df["actual"]

        return forecast_df

    def predict(self):
        t_full_hours = (self.df.index - self.df.index[0]).total_seconds() / 3600
        y_full_fit = self.model_func(t_full_hours, *self.params)
        return t_full_hours, y_full_fit

    def plot(self):
        t_full_hours, y_full_fit = self.predict()

        plt.figure(figsize=(10, 6))
        plt.plot(
            self.df_fit.index, self.df_fit[self.ycol], "o", markersize=3, label="Data"
        )
        # plt.plot(self.df_pred.index, self.df_pred[self.ycol], 'o', markersize=3, color='orange', label='Forecast Data (Actual)')
        plt.plot(self.df.index, y_full_fit, "-", color="red", label="Fitted Model")
        plt.axvline(
            pd.Timestamp((sunset_time - timedelta(days=0)).isot).tz_localize(
                "America/Santiago"
            ),
            color="k",
            ls="--",
            label="Nautical Twilight",
        )

        for i in range(1, 4):
            plt.axvline(
                pd.Timestamp((sunset_time - timedelta(days=i)).isot).tz_localize(
                    "America/Santiago"
                ),
                color="k",
                ls="--",
            )
        plt.xlabel("Time")
        plt.ylabel("Mean Temperature (°C)")
        plt.title("Mean Temperature with Fourier Model Fit")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_residuals(self):
        self.start_date.hour + self.start_date.minute / 60.0
        df_predict = self.df_pred
        t_predict_hours = (df_predict.index - self.df.index[0]).total_seconds() / 3600
        y_predict = self.model_func(t_predict_hours, *self.params)

        residuals = df_predict[self.ycol].values - y_predict
        hour_day = df_predict.index.strftime("%H:%M")

        nautical_index = np.interp(
            self.nautical.timestamp(),  # Convert nautical time to timestamp
            [ts.timestamp() for ts in df_predict.index],  # Convert index to timestamps
            range(len(df_predict)),  # Numeric indices of the dataframe
        )

        # Create a figure and axis object
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.axhline(0, color="gray", linestyle="--")
        # Scatter plot
        ax.scatter(
            hour_day, residuals, color="orange", label="Residuals (Prediction - Actual)"
        )
        # Add a vertical line for nautical twilight
        ax.axvline(nautical_index, label="Nautical Twilight", color="k", ls="--")

        # Set custom x-tick intervals
        ax.xaxis.set_major_locator(MaxNLocator(nbins=10))  # Adjust the number of ticks

        ax.set_xlabel("Chile Local Time")
        ax.set_ylabel("Residuals (°C)")
        ax.set_title("Residuals Between Prediction and Actual Data (Last 24 Hours)")
        ax.legend()
        ax.grid(True)
        fig.tight_layout()
        fig.show()

In [ ]:
data = df_outside.copy()
data["mean"] = (data["max"] + data["min"]) / 2.0
data.index = data.index.tz_convert("America/Santiago")

In [ ]:
fourier_fit = FourierFit(data, start_date=f"{day} {start_hour}", ycol="max")
fourier_fit.fit_model()

### Fourier Forecast: Today's Prediction

The input data is sampled every 30min, the prediction have the same frequency. You can check the residual prediction starting from sunrise time up to twilight time.

In [ ]:
forecast_df = fourier_fit.forecast_end_hour(evening_twilight_local_datetime)
print("Table of Forecast of the day")
forecast_df

In [ ]:
forecast_df["hour_minute"] = forecast_df.index.strftime("%H:%M")
nautical_index = np.interp(
    evening_twilight_local_datetime.timestamp(),  # Convert nautical time to timestamp
    [ts.timestamp() for ts in forecast_df.index],  # Convert index to timestamps
    range(len(forecast_df)),  # Numeric indices of the dataframe
)

# Plot residuals vs hour of the day
plt.figure(figsize=(10, 6))
plt.scatter(
    forecast_df["hour_minute"],
    forecast_df["prediction"],
    color="orange",
    label=f"Forecast\nDay: {day}",
)
plt.axhline(
    forecast_df["actual"].iloc[0],
    color="gray",
    linestyle="--",
    linewidth=3,
    label="Acutal Value",
)  # Add a baseline at y=0
plt.axvline(nautical_index, color="k", ls="--", label="Nautical Twilight")

# Format the x-axis
plt.xticks(
    range(0, len(forecast_df["hour_minute"]), 2),
    forecast_df["hour_minute"][::2],
    rotation=45,
)

# Add labels, title, and legend
plt.xlabel("Chile Local Time (HH:MM)", fontsize=14)
plt.ylabel(r"Temperature [$^\circ$ Deg Celsius]", fontsize=14)
plt.title("Forecast Evolution", fontsize=14)
plt.legend(fontsize=14, loc=0)

# Adjust layout and show plot
plt.tight_layout()
plt.show()

### About the Model

The model is basically three cycles variations of 24, 12 and 8 hours plus a trend, i.e.:

$$
T(t) = Season(t, 24\text{h cycle}) + Season(t, 12\text{h cycle}) + Season(t, \text{h cycle}) + Trend(t) 
$$

where $t$ is the elapsed time in hours since the `start_date`. Also the $Season(t, cycle freq)$ is a sum of sin and cos with normalization parameters, for example:
$$
Season(t, 24h) = A_0 \cos(t/24) + B_0 \sin(t/24)
$$

The trend is a second-order polynomial:
$$
Trend(t) = c_0 + c_1 t + c_2 t^2 
$$

The set of free-parameters of this model are nine: ${A_{0/1/2}, B_{0/1/2}\text{ and }C_{0/1/2}}$

### Check The Fitted Model

In [ ]:
print("Plotting the fit and the data")
fourier_fit.plot()
fourier_fit.plot_residuals()